# 🍲 [Day 11](https://adventofcode.com/2020/day/11)

In [1]:
import numpy as np
from collections import defaultdict

def parse(inputs):
    """Returns two binry array, one marking the position of seats,
       the other marking whether seats are occupied"""
    seats = np.array([[0 if c == '.' else 1 if c == 'L' else 2 
                       for c in line] 
                      for line in inputs], 
                     dtype=np.int32)
    seats_mask = seats > 0
    occupied_seats = seats // 2
    return seats_mask, occupied_seats

def conv2d(arr, f):
    """Simple 2d convolution"""
    kw, kh = f.shape
    kwl = int(np.floor(kw // 2))
    khb = int(np.floor(kh // 2))
    conv_out = np.zeros_like(arr)
    # Padding
    arr_p = np.pad(arr, ((kwl, kw - kwl), (khb, kh - khb)))
    for i in range(conv_out.shape[0]):
        for j in range(conv_out.shape[1]):
            conv_out[i, j] = np.sum(arr_p[i:i + kw, j:j + kh] * f)
    return conv_out
    

def game_of_seats(init_occupied_seats, seats_mask):
    """Update the seats states according to Part 1 until equilibrium"""
    # Filter for convolution
    f = np.ones((3, 3))
    f[1, 1] = 0
    # Apply game of life rules until finding a loop
    seats = np.array(init_occupied_seats)
    while 1:
        previous_seats = np.array(seats)
        # compute number of adjacent
        adjacent = conv2d(seats, f)
        # Apply rule to all seats simultaneously
        seats = ((1 - seats) * seats_mask * (adjacent == 0) +  # Empty seat becomes occupied if no adjacent
                 seats * (adjacent < 4)           # Occupied seat becomes free if >=4 adjacent
                ) 
        # Check if we found a loop
        if np.sum(seats != previous_seats) == 0:
            return np.sum(seats)
        
        
def build_neighbors_mask(seats_mask):
    """Build a neighbor graph that ignores empty/no seats positions."""
    neighbors = defaultdict(lambda: [])
    # Add neighbors along rows
    for i in range(seats_mask.shape[0]):
        last_seat = None
        for j in range(seats_mask.shape[1]):
            if seats_mask[i, j]:
                if last_seat is not None:
                    neighbors[(i, j)].append((i, last_seat))
                    neighbors[(i, last_seat)].append((i, j))
                last_seat = j
    # Add neighbors along columns
    for j in range(seats_mask.shape[1]):
        last_seat = None
        for i in range(seats_mask.shape[0]):
            if seats_mask[i, j]:
                if last_seat is not None:
                    neighbors[(i, j)].append((last_seat, j))
                    neighbors[(last_seat, j)].append((i, j))
                last_seat = i
    # Add neighbors along up-diagonals
    for i in range(seats_mask.shape[0]):
        # Up-diagonal
        j, last_seat = 0, None
        while i >= 0 and j < seats_mask.shape[1]:
            if seats_mask[i, j]:
                if last_seat is not None:
                    neighbors[(i, j)].append(last_seat)
                    neighbors[last_seat].append((i, j))
                last_seat = (i, j)
            i -= 1
            j += 1
    for j in range(1, seats_mask.shape[1]):
        i, last_seat = seats_mask.shape[0] - 1, None
        while i >= 0 and j < seats_mask.shape[1]:
            if seats_mask[i, j]:
                if last_seat is not None:
                    neighbors[(i, j)].append(last_seat)
                    neighbors[last_seat].append((i, j))
                last_seat = (i, j)
            i -= 1
            j += 1
    # Add neighbors along down-diagonals
    for i in range(seats_mask.shape[0]):
        j, last_seat = 0, None
        while i < seats_mask.shape[0] and j < seats_mask.shape[1]:
            if seats_mask[i, j]:
                if last_seat is not None:
                    neighbors[(i, j)].append(last_seat)
                    neighbors[last_seat].append((i, j))
                last_seat = (i, j)
            i += 1
            j += 1
    for j in range(1, seats_mask.shape[1]):
        i, last_seat = 0, None
        while i < seats_mask.shape[0] and j < seats_mask.shape[1]:
            if seats_mask[i, j]:
                if last_seat is not None:
                    neighbors[(i, j)].append(last_seat)
                    neighbors[last_seat].append((i, j))
                last_seat = (i, j)
            i += 1
            j += 1
    return neighbors


def game_of_seats_part2(init_occupied_seats, seats_mask):
    """Update the seats states according to Part 2 until equilibrium"""
    neighbors = build_neighbors_mask(seats_mask)
    seats = {(x, y): init_occupied_seats[x, y] for (x, y) in neighbors}
    # Up
    while 1:
        adjencies = {k: sum(seats[x, y] for (x, y) in neighbors[k]) for k in neighbors}
        change = False
        
        # Update seats positions
        for (x, y) in seats:
            if seats[x, y] == 0 and adjencies[x, y] == 0:
                change = True
                seats[x, y] = 1
            elif seats[x, y] == 1 and adjencies[x, y] >= 5:
                change = True
                seats[x, y] = 0
                
        # If no change, we reached equilibrium
        if not change:
            break
    return sum(seats.values())

In [2]:
%%time
with open('inputs/day11.txt', 'r') as f:
    seats_mask, occupied_seats = parse(f.read().splitlines())
    
print(f"There are {game_of_seats(occupied_seats, seats_mask)} occupied seats at equilibrium")
print(f"Under the new rules, there are {game_of_seats_part2(occupied_seats, seats_mask)} "
      "occupied seats at equilibrium")
print()

There are 2152 occupied seats at equilibrium
Under the new rules, there are 1937 occupied seats at equilibrium

CPU times: user 12.8 s, sys: 37.1 ms, total: 12.8 s
Wall time: 12.9 s
